In [ ]:
pip install XlsxWriter
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
# Lê o arquivo CSV
df = pd.read_csv('C:/Teste_Python/Onix_Apriori.csv', sep = ';')

In [ ]:
# Normaliza os caracteres da coluna "Descricao_Peca_Padrao"
df['Descricao_Peca_Padrao'] = df['Descricao_Peca_Padrao'].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
# Regras para LATERAL DIANTEIRA ESQUERDA
# selecao_lde = (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA ESQUERDA')
# transacoes  = df.loc[selecao_lde, ['Id_Orcamento', 'Descricao_Peca_Padrao']].groupby(['Id_Orcamento'])['Descricao_Peca_Padrao'].apply(list).values.tolist()

In [ ]:
# Regras para LATERAL DIANTEIRA DIREITA
# selecao_ldd = (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA DIREITA')
# transacoes  = df.loc[selecao_ldd, ['Id_Orcamento', 'Descricao_Peca_Padrao']].groupby(['Id_Orcamento'])['Descricao_Peca_Padrao'].apply(list).values.tolist()

In [ ]:
# Regras para DIANTEIRA
# selecao_diant = (df['Regiao_Afetada'] == 'DIANTEIRA')
# transacoes  = df.loc[selecao_diant, ['Id_Orcamento', 'Descricao_Peca_Padrao']].groupby(['Id_Orcamento'])['Descricao_Peca_Padrao'].apply(list).values.tolist()

In [ ]:
# Regras para LATERAL DIANTEIRA ESQUERDA e LATERAL DIANTEIRA DIREITA
# selecao_lde_ldd = (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA ESQUERDA') | (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA DIREITA')
# transacoes  = df.loc[selecao_lde_ldd, ['Id_Orcamento', 'Descricao_Peca_Padrao']].groupby(['Id_Orcamento'])['Descricao_Peca_Padrao'].apply(list).values.tolist()

In [ ]:
# Regras para LATERAL DIANTEIRA ESQUERDA e DIANTEIRA
# selecao_lde_diant = (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA ESQUERDA') | (df['Regiao_Afetada'] == 'DIANTEIRA')
# transacoes  = df.loc[selecao_lde_diant, ['Id_Orcamento', 'Descricao_Peca_Padrao']].groupby(['Id_Orcamento'])['Descricao_Peca_Padrao'].apply(list).values.tolist()

In [ ]:
# Regras para LATERAL DIANTEIRA DIREITA e DIANTEIRA
# selecao_ldd_diant = (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA DIREITA') | (df['Regiao_Afetada'] == 'DIANTEIRA')
# transacoes  = df.loc[selecao_ldd_diant, ['Id_Orcamento', 'Descricao_Peca_Padrao']].groupby(['Id_Orcamento'])['Descricao_Peca_Padrao'].apply(list).values.tolist()

In [ ]:
# Regras para LATERAL DIANTEIRA ESQUERDA, LATERAL DIANTEIRA DIREITA e DIANTEIRA
# selecao_lde_ldd_diant = (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA ESQUERDA') | (df['Regiao_Afetada'] == 'LATERAL DIANTEIRA DIREITA') | (df['Regiao_Afetada'] == 'DIANTEIRA')
# transacoes  = df.loc[selecao_lde_ldd_diant, ['Id_Orcamento', 'Descricao_Peca_Padrao']].groupby(['Id_Orcamento'])['Descricao_Peca_Padrao'].apply(list).values.tolist()

In [ ]:
# Cria o encoder e transforma as transações em binário
encoder = TransactionEncoder()
transacoes_binarias = encoder.fit_transform(transacoes)

In [ ]:
# Cria o dataframe das regras de associação
df_regras = apriori(pd.DataFrame(transacoes_binarias, columns=encoder.columns_), min_support=0.06, use_colnames=True)

In [ ]:
# Filtra as regras de acordo com o lift mínimo
regras_filtradas = association_rules(df_regras, metric="lift", min_threshold=1)

In [ ]:
# Define os consequentes desejados
itens_desejados = [
    'caixa roda dianteira esq',
    'caixa roda dianteira dir',
    'longarina parcial dianteira esquerda',
    'longarina parcial dianteira direita'
]

In [ ]:
regras_selecionadas = regras_filtradas[
    regras_filtradas['consequents'].apply(lambda consequents: any(x in consequents for x in itens_desejados))
]

In [ ]:
# Categorização das regras
categoria = []
for index, row in regras_selecionadas.iterrows():
    antecedentes = list(row['antecedents'])
    consequentes = list(row['consequents'])
    num_itens_desejados = sum(item in antecedentes or item in consequentes for item in itens_desejados)
    if num_itens_desejados == 1:
        categoria.append("Pequena Monta")
    else:
        categoria.append("Media Monta")

In [ ]:
# Adiciona a coluna de categoria
regras_selecionadas = regras_selecionadas.copy()
regras_selecionadas['Categoria'] = categoria

In [ ]:
# Exibe as regras de associação selecionadas
num_total_regras = len(regras_selecionadas)
print("Regras de Associação com Pelo Menos um dos Itens Desejados nos Antecedentes e Consequentes:")
for index, row in regras_selecionadas.iterrows():
    antecedentes = list(row['antecedents'])
    consequentes = list(row['consequents'])
    print("Se", antecedentes, "então", consequentes, "(suporte:", round(row['support'], 3),
          ", confiança:", round(row['confidence'], 3), ", lift:", round(row['lift'], 3),
          ", categoria:", row['Categoria'], ")")

print("Total de", num_total_regras,
      "regras de associação encontradas com pelo menos um dos itens desejados nos antecedentes e consequentes.")

In [ ]:
# Salva as regras de associação em um arquivo XLSX
regras_selecionadas.to_excel('LatDir_LatEsq_Diant.xlsx', index=False)